<a href="https://colab.research.google.com/github/Rosefinch-Midsummer/Awesome-Colab/blob/master/fastai/02_fully_connected%2Bwhy_sqrt5%2Binitializing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
!git clone https://github.com/fastai/course-v3

Cloning into 'course-v3'...
remote: Enumerating objects: 5333, done.
remote: Total 5333 (delta 0), reused 0 (delta 0), pack-reused 5333
Receiving objects: 100% (5333/5333), 257.77 MiB | 32.89 MiB/s, done.
Resolving deltas: 100% (2885/2885), done.


In [6]:
cd /content/course-v3/nbs/dl2

/content/course-v3/nbs/dl2


## The forward and backward passes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=4960)

In [0]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [8]:
x_train,y_train,x_valid,y_valid = get_data()

In [9]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [0]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [11]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [0]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [0]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [14]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [16]:
x_valid.shape

torch.Size([10000, 784])

In [17]:
y_valid.shape

torch.Size([10000])

In [18]:
y_valid.max()

tensor(9)

## Foundations version

### Basic architecture

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [0]:
# num hidden
nh = 50

[Tinker practice](https://course.fast.ai/videos/?lesson=8&t=5255)

In [0]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [0]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [21]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [0]:
def lin(x, w, b): return x@w + b

In [0]:
t = lin(x_valid, w1, b1)

In [25]:
x_valid.shape, w1.shape, b1.shape

(torch.Size([10000, 784]), torch.Size([784, 50]), torch.Size([50]))

In [26]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.1962), tensor(1.0316))

In [0]:
def relu(x): return x.clamp_min(0.)

In [0]:
t = relu(lin(x_valid, w1, b1))

In [30]:
t

tensor([[2.0389, 0.6847, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.6182, 0.1870, 0.0000,  ..., 0.0089, 0.0000, 0.5504],
        [0.9964, 1.2447, 0.2203,  ..., 0.8462, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.2521],
        [0.3192, 0.9287, 0.0000,  ..., 2.5437, 0.0186, 0.0000],
        [0.6174, 0.1591, 0.0000,  ..., 0.1295, 0.0232, 1.0814]])

In [29]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.5121), tensor(0.6780))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [0]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [32]:
w1.mean(),w1.std()

(tensor(-0.0004), tensor(0.0504))

In [33]:
w1

tensor([[ 0.0307,  0.0263,  0.0058,  ..., -0.0108, -0.0655, -0.0601],
        [-0.1178,  0.0230,  0.0255,  ..., -0.0050,  0.0316,  0.0131],
        [-0.0432,  0.0456, -0.0202,  ...,  0.0653,  0.0482,  0.0724],
        ...,
        [ 0.0379,  0.0391,  0.0139,  ...,  0.0675, -0.1303,  0.0077],
        [-0.0784,  0.0142, -0.0533,  ...,  0.0334,  0.0378,  0.0006],
        [-0.0557, -0.0444, -0.1293,  ...,  0.1369, -0.0269,  0.0746]])

In [34]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.6619), tensor(0.8602))

In [0]:
#export
from torch.nn import init

In [0]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [0]:
init.kaiming_normal_??

In [38]:
w1.mean(),w1.std()

(tensor(-0.0004), tensor(0.0505))

In [39]:
t.mean(),t.std()

(tensor(0.5509), tensor(0.7888))

In [40]:
w1.shape

torch.Size([784, 50])

In [0]:
import torch.nn

In [42]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [48]:
torch.nn.Linear(m,nh).weight

Parameter containing:
tensor([[ 0.0102,  0.0014,  0.0124,  ..., -0.0056, -0.0319,  0.0220],
        [-0.0115,  0.0276, -0.0328,  ...,  0.0156,  0.0194,  0.0166],
        [-0.0200, -0.0324,  0.0326,  ..., -0.0092, -0.0062,  0.0129],
        ...,
        [ 0.0012, -0.0194, -0.0095,  ..., -0.0288,  0.0314, -0.0088],
        [ 0.0052, -0.0281,  0.0287,  ...,  0.0147,  0.0344, -0.0222],
        [-0.0062,  0.0226, -0.0320,  ...,  0.0334,  0.0307, -0.0325]],
       requires_grad=True)

In [49]:
torch.nn.Linear(m,nh).bias

Parameter containing:
tensor([-0.0280,  0.0007,  0.0057,  0.0237, -0.0288, -0.0204, -0.0079,  0.0331,
        -0.0051, -0.0271, -0.0072, -0.0012, -0.0036,  0.0293, -0.0327,  0.0214,
         0.0276,  0.0129, -0.0241,  0.0304,  0.0227, -0.0066, -0.0314, -0.0172,
        -0.0231,  0.0133, -0.0335, -0.0300, -0.0095,  0.0032, -0.0325,  0.0295,
         0.0191,  0.0173,  0.0022,  0.0344,  0.0336,  0.0147,  0.0039, -0.0325,
        -0.0319, -0.0016, -0.0129,  0.0330, -0.0002, -0.0266, -0.0310, -0.0113,
         0.0218, -0.0100], requires_grad=True)

In [0]:
torch.nn.Linear.forward??

In [0]:
torch.nn.functional.linear??

In [0]:
torch.nn.Conv2d??

In [0]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [0]:
# what if...?
def relu(x): return x.clamp_min(0.) - 0.5

In [50]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.0687), tensor(0.8393))

In [0]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [52]:
%timeit -n 10 _=model(x_valid)

10 loops, best of 3: 20.3 ms per loop


In [0]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6372)

In [54]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [0]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [0]:
y_train,y_valid = y_train.float(),y_valid.float()

In [0]:
preds = model(x_train)

In [58]:
preds.shape

torch.Size([50000, 1])

In [59]:
mse(preds, y_train)

tensor(34.4383)

### Gradients and backward pass

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6493)

In [0]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [0]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [0]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [0]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [0]:
forward_and_backward(x_train, y_train)

In [0]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [0]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [0]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [0]:
loss = forward(xt2, y_train)

In [0]:
loss.backward()

In [0]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7112)

In [0]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [0]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [0]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [0]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [76]:
%time loss = model(x_train, y_train)

CPU times: user 112 ms, sys: 0 ns, total: 112 ms
Wall time: 111 ms


In [77]:
%time model.backward()

CPU times: user 3.67 s, sys: 4.04 s, total: 7.72 s
Wall time: 7.73 s


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [0]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [0]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [0]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [0]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [0]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [85]:
%time loss = model(x_train, y_train)

CPU times: user 111 ms, sys: 2.32 ms, total: 113 ms
Wall time: 114 ms


In [86]:
%time model.backward()

CPU times: user 232 ms, sys: 19.2 ms, total: 251 ms
Wall time: 255 ms


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7484)

In [0]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [90]:
%time loss = model(x_train, y_train)

CPU times: user 106 ms, sys: 230 µs, total: 107 ms
Wall time: 108 ms


In [91]:
%time model.backward()

CPU times: user 217 ms, sys: 1.44 ms, total: 218 ms
Wall time: 218 ms


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [0]:
#export
from torch import nn

In [0]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [0]:
model = Model(m, nh, 1)

In [96]:
%time loss = model(x_train, y_train)

CPU times: user 96.9 ms, sys: 871 µs, total: 97.8 ms
Wall time: 105 ms


In [97]:
%time loss.backward()

CPU times: user 78.7 ms, sys: 1.4 ms, total: 80.1 ms
Wall time: 80.9 ms


## Export

In [0]:
#!./notebook2script.py 02_fully_connected.ipynb

# why_sqrt5

## Does nn.Conv2d init work well?

In [0]:
#export
from exp.nb_02 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [0]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [101]:
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)
x_train.shape,x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [102]:
n,*_ = x_train.shape
c = y_train.max()+1
nh = 32
n,c

(50000, tensor(10))

In [103]:
l1 = nn.Conv2d(1, nh, 5)
x = x_valid[:100]
x.shape

torch.Size([100, 1, 28, 28])

In [104]:
l1

Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))

In [105]:
def stats(x): return x.mean(),x.std()

l1.weight.shape

torch.Size([32, 1, 5, 5])

In [106]:
stats(l1.weight),stats(l1.bias)

((tensor(0.0006, grad_fn=<MeanBackward0>),
  tensor(0.1175, grad_fn=<StdBackward0>)),
 (tensor(-0.0204, grad_fn=<MeanBackward0>),
  tensor(0.1059, grad_fn=<StdBackward0>)))

In [107]:
t = l1(x)

stats(t)

(tensor(-0.0190, grad_fn=<MeanBackward0>),
 tensor(0.6283, grad_fn=<StdBackward0>))

In [108]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(-0.0226, grad_fn=<MeanBackward0>),
 tensor(1.0758, grad_fn=<StdBackward0>))

In [109]:
import torch.nn.functional as F
def f1(x,a=0): 
  return F.leaky_relu(l1(x),a)

init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5465, grad_fn=<MeanBackward0>),
 tensor(1.0927, grad_fn=<StdBackward0>))

In [110]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.1965, grad_fn=<MeanBackward0>),
 tensor(0.3900, grad_fn=<StdBackward0>))

In [111]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [112]:
# receptive field size
rec_fs = l1.weight[0,0].numel()
rec_fs

25

In [113]:
nf,ni,*_ = l1.weight.shape
nf,ni

(32, 1)

In [114]:
fan_in  = ni*rec_fs
fan_out = nf*rec_fs
fan_in,fan_out

(25, 800)

In [115]:
def gain(a): return math.sqrt(2.0 / (1 + a**2))

gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

In [116]:
torch.zeros(10000).uniform_(-1,1).std()

tensor(0.5818)

In [117]:
1/math.sqrt(3.)

0.5773502691896258

In [118]:
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

kaiming2(l1.weight, a=0);
stats(f1(x))


(tensor(0.5341, grad_fn=<MeanBackward0>),
 tensor(1.0294, grad_fn=<StdBackward0>))

In [119]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.2267, grad_fn=<MeanBackward0>),
 tensor(0.4029, grad_fn=<StdBackward0>))

In [0]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

In [0]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [122]:
y = y_valid[:100].float()
y

tensor([3., 8., 6., 9., 6., 4., 5., 3., 8., 4., 5., 2., 3., 8., 4., 8., 1., 5.,
        0., 5., 9., 7., 4., 1., 0., 3., 0., 6., 2., 9., 9., 4., 1., 3., 6., 8.,
        0., 7., 7., 6., 8., 9., 0., 3., 8., 3., 7., 7., 8., 4., 4., 1., 2., 9.,
        8., 1., 1., 0., 6., 6., 5., 0., 1., 1., 7., 2., 7., 3., 1., 4., 0., 5.,
        0., 6., 8., 7., 6., 8., 9., 9., 4., 0., 6., 1., 9., 2., 6., 3., 9., 4.,
        4., 5., 6., 6., 1., 7., 2., 8., 6., 9.])

In [123]:
t = m(x)
stats(t)

(tensor(0.0473, grad_fn=<MeanBackward0>),
 tensor(0.0107, grad_fn=<StdBackward0>))

In [0]:
l = mse(t,y)
l.backward()

In [125]:
stats(m[0].weight.grad)

(tensor(-0.0009), tensor(0.0289))

In [0]:
init.kaiming_uniform_??

In [127]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

t = m(x)
stats(t)

(tensor(-0.0576, grad_fn=<MeanBackward0>),
 tensor(0.3668, grad_fn=<StdBackward0>))

In [128]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(-0.2113), tensor(0.8455))

## Export

In [0]:
#!./notebook2script.py 02a_why_sqrt5.ipynb

# initializing

## Why you need a good init

To understand why initialization is important in a neural net, we'll focus on the basic operation you have there: matrix multiplications. So let's just take a vector x, and a matrix a initiliazed randomly, then multiply them 100 times (as if we had 100 layers).

In [130]:
import torch
x = torch.randn(512)
a = torch.randn(512,512)

for i in range(100): x = a @ x

x.mean(),x.std()

(tensor(nan), tensor(nan))

In [131]:
#The problem you'll get with that is activation explosion: very soon, your activations will go to nan. We can even ask the loop to break when that first happens:

x = torch.randn(512)
a = torch.randn(512,512)

for i in range(100): 
    x = a @ x
    if x.std() != x.std(): break

i

27

In [132]:
#It only takes 27 multiplications! On the other hand, if you initialize your activations with a scale that is too low, then you'll get another problem:

x = torch.randn(512)
a = torch.randn(512,512) * 0.01

for i in range(100): x = a @ x

x.mean(),x.std()

(tensor(0.), tensor(0.))

Here, every activation vanished to 0. So to avoid that problem, people have come with several strategies to initialize their weight matices, such as:

- use a standard deviation that will make sure x and Ax have exactly the same scale
- use an orthogonal matrix to initialize the weight (orthogonal matrices have the special property that they preserve the L2 norm, so x and Ax would have the same sum of squares in that case)
- use [spectral normalization](https://arxiv.org/pdf/1802.05957.pdf) on the matrix A (the spectral norm of A is the least possible number M such that torch.norm(A@x) <= M*torch.norm(x) so dividing A by this M insures you don't overflow. You can still vanish with this)

## The magic number for scaling

Here we will focus on the first one, which is the Xavier initialization. It tells us that we should use a scale equal to 1/math.sqrt(n_in) where n_in is the number of inputs of our matrix.

In [133]:
import math

x = torch.randn(512)
a = torch.randn(512,512) / math.sqrt(512)

for i in range(100): x = a @ x

x.mean(),x.std()

(tensor(0.3030), tensor(6.3517))

In [134]:
#And indeed it works. Note that this magic number isn't very far from the 0.01 we had earlier.

1/ math.sqrt(512)

0.044194173824159216

But where does it come from? It's not that mysterious if you remember the definition of the matrix multiplication. When we do y = a @ x, the coefficients of y are defined by

$$y_{i} = a_{i,0} x_{0} + a_{i,1} x_{1} + \cdots + a_{i,n-1} x_{n-1} = \sum_{k=0}^{n-1} a_{i,k} x_{k}$$
or in code:

y[i] = sum([c*d for c,d in zip(a[i], x)])

Now at the very beginning, our x vector has a mean of roughly 0. and a standard deviation of roughly 1. (since we picked it that way).

In [135]:
x = torch.randn(512)
x.mean(), x.std()

(tensor(-0.0486), tensor(1.0000))

NB: This is why it's extremely important to normalize your inputs in Deep Learning, the intialization rules have been designed with inputs that have a mean 0. and a standard deviation of 1.

If you need a refresher from your statistics course, the mean is the sum of all the elements divided by the number of elements (a basic average). The standard deviation represents if the data stays close to the mean or on the contrary gets values that are far away. It's computed by the following formula:

$$\sigma = \sqrt{\frac{1}{n}\left[(x_{0}-m)^{2} + (x_{1}-m)^{2} + \cdots + (x_{n-1}-m)^{2}\right]}$$
where m is the mean and $\sigma$ (the greek letter sigma) is the standard deviation. Here we have a mean of 0, so it's just the square root of the mean of x squared.

If we go back to y = a @ x and assume that we chose weights for a that also have a mean of 0, we can compute the standard deviation of y quite easily. Since it's random, and we may fall on bad numbers, we repeat the operation 100 times.

In [136]:
mean,sqr = 0.,0.
for i in range(100):
    x = torch.randn(512)
    a = torch.randn(512, 512)
    y = a @ x
    mean += y.mean().item()
    sqr  += y.pow(2).mean().item()
mean/100,sqr/100

(0.04189484797418117, 520.1733755493165)

In [137]:
mean

4.1894847974181175

In [138]:
sqr

52017.33755493164

In [139]:
y.mean()

tensor(2.5895)

In [140]:
y.mean().item()

2.5894575119018555

Now that looks very close to the dimension of our matrix 512. And that's no coincidence! When you compute y, you sum 512 product of one element of a by one element of x. So what's the mean and the standard deviation of such a product? We can show mathematically that as long as the elements in a and the elements in x are independent, the mean is 0 and the std is 1. This can also be seen experimentally:

In [141]:
mean,sqr = 0.,0.
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)
    y = a*x
    mean += y.item()
    sqr  += y.pow(2).item()
mean/10000,sqr/10000

(0.0009153877577288313, 1.0218506309398483)

Then we sum 512 of those things that have a mean of zero, and a mean of squares of 1, so we get something that has a mean of 0, and mean of square of 512, hence math.sqrt(512) being our magic number. If we scale the weights of the matrix a and divide them by this math.sqrt(512), it will give us a y of scale 1, and repeating the product has many times as we want won't overflow or vanish.

## Adding ReLU in the mix

We can reproduce the previous experiment with a ReLU, to see that this time, the mean shifts and the standard deviation becomes 0.5. This time the magic number will be math.sqrt(2/512) to properly scale the weights of the matrix.

In [142]:
mean,sqr = 0.,0.
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)
    y = a*x
    y = 0 if y < 0 else y.item()
    mean += y
    sqr  += y ** 2
mean/10000,sqr/10000

(0.3171515812907399, 0.49948872303744063)

In [143]:
#We can double check by running the experiment on the whole matrix product.

mean,sqr = 0.,0.
for i in range(100):
    x = torch.randn(512)
    a = torch.randn(512, 512)
    y = a @ x
    y = y.clamp(min=0)
    mean += y.mean().item()
    sqr  += y.pow(2).mean().item()
mean/100,sqr/100

(9.008800077438355, 255.70396621704103)

In [144]:
#Or that scaling the coefficient with the magic number gives us a scale of 1.

mean,sqr = 0.,0.
for i in range(100):
    x = torch.randn(512)
    a = torch.randn(512, 512) * math.sqrt(2/512)
    y = a @ x
    y = y.clamp(min=0)
    mean += y.mean().item()
    sqr  += y.pow(2).mean().item()
mean/100,sqr/100

(0.5599711841344833, 0.9914185112714767)

The math behind is a tiny bit more complex, and you can find everything in the [Kaiming](https://arxiv.org/abs/1502.01852) and the [Xavier](http://proceedings.mlr.press/v9/glorot10a.html) paper but this gives the intuition behing those results.